In [12]:
from PIL import Image
import aiohttp
import asyncio
from io import BytesIO
import gspread

METADATA_BYTES = 1024
BATCH_SIZE = 10000
GS_MAX_ROW = 46900
MAX_SESSION_COUNT = 150

path_to_json = "/home/helga/Downloads/cedar-router-404907-8247ccef8427.json"
path_to_spreadsheet = "https://docs.google.com/spreadsheets/d/1rtsWoauhfEmOld6oXEVjbQul4tFNVZwnurCn3O4NNJo/edit#gid=1902149593"

gc = gspread.service_account(path_to_json)
spreadsheet = gc.open_by_url(path_to_spreadsheet)

worksheet = spreadsheet.get_worksheet(0)
session_count = 0
print("Process started.")


async def get_size(url, p_dict, p_index):
    global session_count
    headers = {
        'Referer': 'https://example.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
    }
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(url, headers=headers) as response:
                raw = await response.content.read(METADATA_BYTES)
                response.raise_for_status()  # Raise an exception for bad status codes
                image = Image.open(BytesIO(raw))
                p_dict[p_index] = f"{image.width}x{image.height}"
    except Exception as e:
        p_dict[p_index] = str(e)
    session_count = session_count - 1


async def run_all():
    global session_count
    size_dict = {}
    i = 1
    end = 1

    task_list = []
    run_process = True
    while run_process:
        start = end + 1
        end = start + BATCH_SIZE - 1
        if end > GS_MAX_ROW:
            end = GS_MAX_ROW
        print(f"Processing range A{start}:A{end}.")
        print(f"i is: {i}")
        for record in worksheet.get(f"A{start}:A{end}"):
            if len(record) == 0:
                print(f"Aborting process as empty list received. i: {i}.")
                run_process = False
                break
            extracted_url = record[0]
            session_count = session_count + 1
            task_list.append(asyncio.create_task(get_size(extracted_url, size_dict, i)))
            i = i + 1
            if i == GS_MAX_ROW:
                print(f"Aborting process as maximum row index received. i: {i}.")
                run_process = False
                break

            if session_count >= MAX_SESSION_COUNT:
                await asyncio.sleep(0.25)

    await asyncio.gather(*task_list)
    return size_dict


size_dict = (await run_all())
size_list = [[str(size_dict[key])] for key in sorted(size_dict.keys())]
cell_cnt = len(size_list)

worksheet.batch_update([{"range": "B2", "values": size_list}])
print("The process has successfully finished.")

Process started.
Processing range A2:A10001.
i is: 1
Processing range A10002:A20001.
i is: 10001
Processing range A20002:A30001.
i is: 20001
Processing range A30002:A40001.
i is: 30001
Processing range A40002:A46900.
i is: 40001
Processing range A46901:A46900.
i is: 46889
Aborting process as empty list received. i: 46889.
The process has successfully finished.
